In [10]:
# Cell 1: Setup & Imports
import sys
sys.path.insert(0, '/home/savaj.k/.local/lib/python3.10/site-packages')

import os
import shutil
from pathlib import Path
import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Add project root to path
if str(Path.cwd()) not in sys.path:
    sys.path.append(str(Path.cwd()))

# Check GPU
print(f"PyTorch Version: {torch.__version__}")
if torch.cuda.is_available():
    print(f" GPU Detected: {torch.cuda.get_device_name(0)}")
    print(f"   Available GPUs: {torch.cuda.device_count()}")
else:
    print(" No GPU detected!")

# Import project modules
from src.config import (
    RAW_DATA_DIR, DATASET_YAML, WEIGHTS_DIR, 
    CLASS_NAMES, NUM_CLASSES, create_directories, print_config_summary
)
from src.data_utils import verify_dataset_structure, create_data_yaml, print_dataset_summary
from src.pattern_detector import PatternDetector

create_directories()
print_config_summary()

PyTorch Version: 2.5.1
 GPU Detected: Tesla P100-PCIE-12GB
   Available GPUs: 4
PROJECT CONFIGURATION SUMMARY
Project Root: /home/savaj.k/PRCV-Project
Number of Classes: 6
Classes: Dragonfly Doji, Gravestone Doji, Hammer, Hanging Man, Marubozu, Spinning Top
Model Architecture: yolov8s.pt
Training Epochs: 50
Batch Size: 16
Image Size: 640
API Key Set: No


In [11]:
# Cell 2: Verify Dataset
print("=" * 60)
print("STEP 1: DATA VERIFICATION")
print("=" * 60)

if verify_dataset_structure():
    print("\n Dataset found!")
    create_data_yaml()
    print_dataset_summary()
else:
    print("\n Dataset not found! Please extract to data/raw/")

STEP 1: DATA VERIFICATION
Checking dataset structure...
----------------------------------------
  [OK] raw/train/images
  [OK] raw/train/labels
  [OK] raw/valid/images
  [OK] raw/valid/labels
  [OK] raw/data.yaml
  [OK] raw/test/images
  [OK] raw/test/labels
----------------------------------------
Dataset structure verified successfully!

 Dataset found!
Created data.yaml at: /home/savaj.k/PRCV-Project/data/raw/data.yaml

DATASET SUMMARY (6-Class Candlestick Patterns)

Images:
  Training:   4,102
  Validation: 1,175
  Test:       596
  Total:      5,873

Annotated Instances: 5,873

Class Distribution (Training Set):
---------------------------------------------
  Spinning Top          2211 ████████████████████
  Marubozu               648 █████
  Gravestone Doji        470 ████
  Dragonfly Doji         303 ██
  Hammer                 281 ██
  Hanging Man            189 █


In [3]:
# Cell 3: Training
print("=" * 60)
print("STEP 2: MODEL TRAINING")
print("=" * 60)

# Load YOLOv8 Small model
model = YOLO('yolov8s.pt')
print("Loaded YOLOv8-Small")

# Check GPU count
num_gpus = torch.cuda.device_count()
print(f"\n Training with {num_gpus} GPU(s)")

# Training configuration
train_config = {
    'data': str(DATASET_YAML),
    'epochs': 50,
    'patience': 15,
    'batch': 16 * max(1, num_gpus),
    'imgsz': 640,
    'device': list(range(num_gpus)) if num_gpus > 1 else 0,
    'workers': 8,
    'optimizer': 'AdamW',
    'lr0': 0.01,
    'lrf': 0.01,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'box': 7.5,
    'cls': 0.5,
    'dfl': 1.5,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'translate': 0.1,
    'scale': 0.5,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'project': 'models',
    'name': 'candlestick_v1',
    'exist_ok': True,
    'verbose': True,
    'plots': True
}

print(f"\nConfiguration:")
print(f"  - Epochs: {train_config['epochs']}")
print(f"  - Batch Size: {train_config['batch']}")
print(f"  - Image Size: {train_config['imgsz']}")
print("-" * 60)

# Start training
results = model.train(**train_config)
print("\n Training Complete!")

STEP 2: MODEL TRAINING
✅ Loaded YOLOv8-Small

🚀 Training with 4 GPU(s)

Configuration:
  - Epochs: 50
  - Batch Size: 64
  - Image Size: 640
------------------------------------------------------------
New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.234 🚀 Python-3.10.16 torch-2.5.1 CUDA:0 (Tesla P100-PCIE-12GB, 12194MiB)
                                                 CUDA:1 (Tesla P100-PCIE-12GB, 12194MiB)
                                                 CUDA:2 (Tesla P100-PCIE-12GB, 12194MiB)
                                                 CUDA:3 (Tesla P100-PCIE-12GB, 12194MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/savaj.k/PRCV-Project/data/raw/data.yaml, degrees=0.0, determi

In [12]:
# Cell 4: Save Weights & Evaluate
print("=" * 60)
print("STEP 3: SAVE WEIGHTS & EVALUATE")
print("=" * 60)

# Copy best weights to production path
run_weights = Path('models/candlestick_v1/weights/best.pt')
final_weights = WEIGHTS_DIR / 'best.pt'

if run_weights.exists():
    shutil.copy(run_weights, final_weights)
    print(f" Weights saved to: {final_weights}")
    print(f"   Size: {final_weights.stat().st_size / 1e6:.1f} MB")
else:
    print(" Weights not found. Check training logs.")

# Run validation
print("\n Running Validation...")
val_results = model.val(data=str(DATASET_YAML), split='val')

print("\n" + "=" * 60)
print("VALIDATION RESULTS")
print("=" * 60)
print(f"  mAP@50:    {val_results.box.map50:.4f}")
print(f"  mAP@50-95: {val_results.box.map:.4f}")
print(f"  Precision: {val_results.box.mp:.4f}")
print(f"  Recall:    {val_results.box.mr:.4f}")
print("=" * 60)

STEP 3: SAVE WEIGHTS & EVALUATE
 Weights saved to: /home/savaj.k/PRCV-Project/models/weights/best.pt
   Size: 22.5 MB

 Running Validation...
Ultralytics 8.3.234 🚀 Python-3.10.16 torch-2.5.1 CUDA:0 (Tesla P100-PCIE-12GB, 12194MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 48.1±13.8 MB/s, size: 12.2 KB)
val: Scanning /home/savaj.k/PRCV-Project/data/raw/valid/labels.cache... 1175 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1175/1175 1.9Mit/s 0.0s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 74/74 6.6it/s 11.3s0.1s
                   all       1175       1175      0.468      0.735      0.607      0.606
        Dragonfly Doji         95         95      0.408      0.768       0.48      0.477
       Gravestone Doji        144        144      0.504      0.764      0.672      0.671
                Hammer         83         83      0.404      0.639      0.546      0.546
           Hanging Man         52         52 

In [5]:
# Cell 5: Test Detection
print("=" * 60)
print("STEP 4: TEST DETECTION")
print("=" * 60)

from src.config import BULLISH_PATTERNS, BEARISH_PATTERNS

# Load detector
detector = PatternDetector(model_path=WEIGHTS_DIR / 'best.pt')

# Get validation images
valid_images = list((RAW_DATA_DIR / "valid" / "images").glob("*.jpg"))
if not valid_images:
    valid_images = list((RAW_DATA_DIR / "valid" / "images").glob("*.png"))

print(f"Found {len(valid_images)} validation images")

# Test on sample image
if valid_images:
    sample_img = valid_images[0]
    detections, annotated = detector.detect(str(sample_img), confidence_threshold=0.20)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(annotated)
    plt.axis('off')
    plt.title(f"Detected: {len(detections)} patterns")
    plt.show()
    
    detector.print_detections(detections)

STEP 4: TEST DETECTION
Model loaded: best.pt
Found 1175 validation images


<Figure size 1200x800 with 1 Axes>


DETECTED PATTERNS: 1

  1. Spinning Top
     Confidence: 81.0%
     Signal: ⚪ NEUTRAL



In [13]:
# Cell 6: Create Demo Visualization - FIND ALL PATTERN TYPES
print("=" * 60)
print("STEP 5: DEMO VISUALIZATION (All Pattern Types)")
print("=" * 60)

from src.config import BULLISH_PATTERNS, BEARISH_PATTERNS, CLASS_NAMES
from collections import defaultdict

# Find one example of each pattern type
pattern_examples = defaultdict(list)

print("Searching for examples of each pattern...")
for img_path in valid_images:
    dets, annotated = detector.detect(str(img_path), confidence_threshold=0.25)
    
    if dets:
        pattern = dets[0].pattern
        if len(pattern_examples[pattern]) < 2:  # Keep up to 2 examples per pattern
            pattern_examples[pattern].append((img_path, dets, annotated))

print(f"\nFound patterns:")
for pattern, examples in pattern_examples.items():
    print(f"  - {pattern}: {len(examples)} examples")

# Create visualization with one of each pattern
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Get one example per pattern (prioritize variety)
demo_images = []
for pattern in CLASS_NAMES:
    if pattern in pattern_examples and pattern_examples[pattern]:
        demo_images.append(pattern_examples[pattern][0])

# Fill remaining slots with any pattern
for pattern, examples in pattern_examples.items():
    if len(demo_images) >= 6:
        break
    for ex in examples:
        if ex not in demo_images and len(demo_images) < 6:
            demo_images.append(ex)

print(f"\nShowing {len(demo_images)} different patterns in demo")

for idx, ax in enumerate(axes):
    if idx < len(demo_images):
        img_path, dets, annotated = demo_images[idx]
        
        ax.imshow(annotated)
        ax.axis('off')
        
        pattern = dets[0].pattern
        conf = dets[0].confidence
        
        if pattern in BULLISH_PATTERNS:
            color, signal = 'green', '🟢 BULLISH'
        elif pattern in BEARISH_PATTERNS:
            color, signal = 'red', '🔴 BEARISH'
        else:
            color, signal = 'gray', '⚪ NEUTRAL'
        
        ax.set_title(f"{pattern}\n{signal} | {conf:.0%}", fontsize=12, color=color, fontweight='bold')
    else:
        ax.axis('off')

plt.suptitle("YOLOv8 Candlestick Pattern Detection - All 6 Classes", fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig("final_demo.png", dpi=150, bbox_inches='tight')
plt.show()

print("\nDemo saved as 'final_demo.png'")

# Also show per-class summary
print("\n" + "=" * 60)
print("DETECTION SUMMARY BY CLASS")
print("=" * 60)
for pattern in CLASS_NAMES:
    count = len(pattern_examples.get(pattern, []))
    emoji = "🟢" if pattern in BULLISH_PATTERNS else "🔴" if pattern in BEARISH_PATTERNS else "⚪"
    print(f"  {emoji} {pattern}: {count} detections found")
print("=" * 60)

STEP 5: DEMO VISUALIZATION (All Pattern Types)
Searching for examples of each pattern...

Found patterns:
  - Spinning Top: 2 examples
  - Gravestone Doji: 2 examples
  - Dragonfly Doji: 2 examples
  - Hammer: 2 examples
  - Marubozu: 2 examples
  - Hanging Man: 2 examples

Showing 6 different patterns in demo


/tmp/ipykernel_334776/3960105240.py:67: UserWarning: Glyph 128994 (\N{LARGE GREEN CIRCLE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_334776/3960105240.py:67: UserWarning: Glyph 128308 (\N{LARGE RED CIRCLE}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/tmp/ipykernel_334776/3960105240.py:68: UserWarning: Glyph 128994 (\N{LARGE GREEN CIRCLE}) missing from font(s) DejaVu Sans.
  plt.savefig("final_demo.png", dpi=150, bbox_inches='tight')
/tmp/ipykernel_334776/3960105240.py:68: UserWarning: Glyph 128308 (\N{LARGE RED CIRCLE}) missing from font(s) DejaVu Sans.
  plt.savefig("final_demo.png", dpi=150, bbox_inches='tight')


<Figure size 1800x1200 with 6 Axes>


Demo saved as 'final_demo.png'

DETECTION SUMMARY BY CLASS
  🟢 Dragonfly Doji: 2 detections found
  🔴 Gravestone Doji: 2 detections found
  🟢 Hammer: 2 detections found
  🔴 Hanging Man: 2 detections found
  ⚪ Marubozu: 2 detections found
  ⚪ Spinning Top: 2 detections found


In [14]:
# Cell 7: Summary
print("\n" + "=" * 60)
print(" PROJECT COMPLETE!")
print("=" * 60)
print(f"\n Files created:")
print(f"   - Model weights: {WEIGHTS_DIR / 'best.pt'}")
print(f"   - Demo image: final_demo.png")
print(f"\n To run Flask webapp:")
print(f"   cd webapp && python app.py")
print(f"\n Classes trained: {', '.join(CLASS_NAMES)}")
print("=" * 60)


 PROJECT COMPLETE!

 Files created:
   - Model weights: /home/savaj.k/PRCV-Project/models/weights/best.pt
   - Demo image: final_demo.png

 To run Flask webapp:
   cd webapp && python app.py

 Classes trained: Dragonfly Doji, Gravestone Doji, Hammer, Hanging Man, Marubozu, Spinning Top
